# Using reasoning for workflow building

When building out customer service solutions, one of the first steps is around taking knowledge base articles and converting them into a set of routines that an LLM can follow and understand.

Most internal knowledge base articles are complex and designed in such a way that is ideal for humans to parse through and understand. They can oven involve complex diagrams and workflows that would be challenging for an LLM based solution to reason through.

The first step to building out a customer service solution is to take these knowledge base articles and convert them into routines designed specifically for LLMs. This can be a time consuming process and is very challenging for companies to build out an automated pipeline. Automating this process using LLMs helps reduce a lot of the effort spent, but it can often present a new set of challenges in order to tune a model in order to get it right.

o1 has demonstrated the capaiblity to easily break down these articles and convert them into sets of routines optimized for LLMs.

## Selecting Knowledge Base Articles

For this example, we will take a set of publicly available Help Center articles from the OpenAI website, and convert these into internal routines that an LLM would execute on. In addition to converting the policies into routines, we will also have the model generate functions that we can build to have the LLM execute actions on behalf of the user. We will start by using the following help center articles to convert to routines:
- [How do I delete my payment method](https://help.openai.com/en/articles/8156070-how-do-i-delete-my-payment-method)
- [How can I get a Business Associate Agreement (BAA) with OpenAI?](https://help.openai.com/en/articles/8660679-how-can-i-get-a-business-associate-agreement-baa-with-openai)
- [How can I set up prepaid billing?](https://help.openai.com/en/articles/8264644-how-can-i-set-up-prepaid-billing)
- [How do I submit a VAT exemption request](https://help.openai.com/en/articles/7232908-how-do-i-submit-a-vat-exemption-request)

In [78]:
from openai import OpenAI
from IPython.display import display, HTML
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

client = OpenAI()
MODEL = 'o1-preview'

In [68]:
articles = []

In [69]:
articles.append({
    "policy": "Delete Payment Method",
    "content": """
How do I delete my payment method?
Updated over a week ago
We keep your payment method on file to cover any outstanding charges on your account. To stop charges to your payment method, please follow the steps below.

## ChatGPT 
You can cancel your ChatGPT Plus subscription to stop further charges at any time:  
Click on 'My Plan' in the ChatGPT sidebar. 
Click on 'Manage my subscription' in the pop-up window.
Select 'Cancel Plan'. 
Please note that your cancellation will take effect the day after the next billing date, and you can continue using our services until then. To avoid being charged for your next billing period, please cancel your subscription at least 24 hours before your next billing date. 

## API
We'll need to keep a payment method on file to account for any outstanding usage costs. You're welcome to cancel your pay-as-you-go service, by clicking 'Cancel paid account' in your billing overview. After the current month's invoice has been issued, the current card will no longer be charged. 
If you'd like to continue using the service, add a new payment method in the billing overview page and select 'Set as default'. You'll then be able to delete the old payment method.
"""})

In [70]:
articles.append({
    "policy": "Business Associate Agreement",
    "content": """
How can I get a Business Associate Agreement (BAA) with OpenAI?
Information about HIPAA compliance for healthcare companies

The Health Insurance Portability and Accountability Act (HIPAA) is a U.S. federal law that requires privacy and security protections for protected health information (PHI). Our API platform can be a great fit for any covered entity or business associate looking to process protected health information, and we’d be happy to assist you in fulfilling your HIPAA compliance. To use our API platform, you’ll first need a BAA with OpenAI.


How do I get started?
If you require a BAA before you can use our API, email us at baa@openai.com with details about your company and use case.

Our team will respond within 1-2 business days. We review each BAA request on a case-by-case basis and may need additional information. The process is usually completed within a few business days.

                
Can I get a BAA for ChatGPT?
If you're interested in exploring a BAA for ChatGPT Enterprise, please contact sales.
 

What happens if I’m not approved?
We are able to approve most customers that request BAAs, but occasionally a use case doesn’t pass our team's evaluation. In that case, we’ll give feedback and context as to why that is and give you the opportunity to update your intended use of our API and re-apply.

 
Are all API services covered by the BAA?
No, only endpoints that are eligible for zero retention are covered by the BAA. You can see a list of those endpoints. 

 
Is an enterprise agreement requirement to sign a BAA?
No, an enterprise agreement is not required to sign a BAA.
"""})

In [71]:
articles.append({
    "policy": "Set up prepaid billing",
    "content": """

How can I set up prepaid billing?

How it works
Prepaid billing allows API users to pre-purchase usage. The credits you've bought will be applied to your monthly invoice. This means that any API usage you incur will first be deducted from the prepaid credits. If your usage exceeds the credits you've purchased, you'll then be billed for the additional amount.
Prepaid billing helps developers know what they are committing to upfront which can provide more predictability for budgeting and spend management. 


Setting up prepaid billing
If you're on a Monthly Billing plan, you may also choose to switch to prepaid billing and purchase credits upfront for API usage. 
- Go to your billing overview in your account settings
- Click "Start payment plan" (you may see variations like "Buy credits")
    Note: If you previously had an arrears billing plan, you'll need to cancel this existing payment plan first.
- Choose the initial amount of credits you want to purchase. The minimum purchase is $5. The maximum purchase will be based on your trust tier.
- Confirm and purchase your initial amount of credits.
- Use auto-recharge to set an automatic recharge amount, which is the amount of credits that will be added to your account when your balance falls below a set threshold.

Please note that any purchased credits will expire after 1 year and they are non-refundable. 
After you’ve purchased credits, you should be able to start using the API. Note that there may be a couple minutes of delay while our systems update to reflect your credit balance.


Purchasing additional credits
Once you’ve consumed all your credits, your API requests will start returning an error letting you know you’ve hit your billing quota. If you’d like to continue your API usage, you can return to the billing portal and use the “Add to balance” button to purchase additional credits.

 
Delayed billing
Due to the complexity of our billing and processing systems, there may be delays in our ability to cut off access after you consume all of your credits. This excess usage may appear as a negative credit balance in your billing dashboard, and will be deducted from your next credit purchase.
"""})

In [72]:
articles.append({
    "policy": "VAT Exemption request",
    "content": """
How do I submit a VAT exemption request?
Updated over a week ago
If you are eligible for a tax exemption and would like to apply it to your account, please follow these steps: 

Depending on the state and if required: 
1. Obtain a current tax exemption certificate from your state or local government and/or your fully completed non-profit sales tax exemption forms. The certificate/forms should include: 
    Your name and address
    Tax exemption number
    Signatures and dates, etc. 
2. Send us a copy of your certificate using the chat widget in the bottom-right corner. Please include your organization id, invoice number or email address associated with the account, so we can easily find you. Instructions on how to find these items are below.
3. Once we receive your certificate/form, we will review it and apply the tax exemption to your account. You will receive a confirmation email once the exemption has been applied. 

Please note that the tax exemption will only apply to future purchases. We cannot apply VAT exemptions retroactively.

Where to find your account data
In order to submit a Value Added Tax ('VAT') exemption request you will need the following from your organization Billing preferences:
    Company name
    Billing email
    Primary business address
    Business tax ID
"""})

In [73]:
CONVERSION_PROMPT = """
You are a helpful assistant tasked with taking an external facing help center article and converting it into a internal-facing programmatically executable routine optimized for an LLM. 
The LLM using this routine will be tasked with reading the policy, answering incoming questions from customers, and helping drive the case toward resolution.

Please follow these instructions:
1. **Review the customer service policy carefully** to ensure every step is accounted for. It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format.
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
      **Sub-actions should start on new lines**
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
   - **For instructions that require more information from the customer**, provide polite and professional prompts to ask for additional information.
   - **For actions that require data from external systems**, write a step to call a function using backticks for the function name (e.g., `call the check_delivery_date function`).
      - **If a step requires the customer service agent to take an action** (e.g., process a refund), generate a function call for this action (e.g., `call the process_refund function`).
      - **Define any new functions** by providing a brief description of their purpose and required parameters.
   - **If there is an action an assistant can performon behalf of the user**, include a function call for this action (e.g., `call the change_email_address function`), and ensure the function is defined with its purpose and required parameters.
      - This action may not be explicitly defined in the help center article, but can be done to help the user resolve their inquiry faster
   - **The step prior to case resolution should always be to ask if there is anything more you can assist with**.
   - **End with a final action for case resolution**: calling the `case_resolution` function should always be the final step.
4. **Ensure compliance** by making sure all steps adhere to company policies, privacy regulations, and legal requirements.
5. **Handle exceptions or escalations** by specifying steps for scenarios that fall outside the standard policy.

**Important**: If at any point you are uncertain, respond with "I don't know."

Please convert the customer service policy into the formatted routine, ensuring it is easy to follow and execute programmatically.

"""

In [74]:
def generate_routine(policy):
    try:
        messages = [
            {
                "role": "system",
                "content": CONVERSION_PROMPT
            },
            {
                "role": "user",
                "content": policy
            }
        ]

        response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        

        return response.choices[0].message.content 
    except Exception as e:
        print(f"An error occurred: {e}")

In [75]:
def process_article(article):
    routine = generate_routine(article['content'])
    return {"policy": article['policy'], "content": article['content'], "routine": routine}


with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_article, articles))

,policy,content,routine
0,Delete Payment Method,\nHow do I delete my payment method?\nUpdated ...,1. Ask the customer to specify which service t...
1,Business Associate Agreement,\nHow can I get a Business Associate Agreement...,**Instructions:**\n\n1. **Acknowledge the user...
2,Set up prepaid billing,\n\nHow can I set up prepaid billing?\n\nHow i...,1. Greet the customer and acknowledge their re...
3,VAT Exemption request,\nHow do I submit a VAT exemption request?\nUp...,1. Ask the customer to provide a copy of their...


In [82]:
df = pd.DataFrame(results)

# Set display options to show all text in the dataframe cells
pd.set_option('display.max_colwidth', None)

# Function to display formatted text in HTML
def display_formatted_dataframe(df):
    def format_text(text):
        return text.replace('\n', '<br>')

    df_formatted = df.copy()
    df_formatted['content'] = df_formatted['content'].apply(format_text)
    df_formatted['routine'] = df_formatted['routine'].apply(format_text)
    
    display(HTML(df_formatted.to_html(escape=False, justify='left')))

display_formatted_dataframe(df)

,policy,content,routine
0,Delete Payment Method,"How do I delete my payment method?Updated over a week agoWe keep your payment method on file to cover any outstanding charges on your account. To stop charges to your payment method, please follow the steps below.## ChatGPT You can cancel your ChatGPT Plus subscription to stop further charges at any time: Click on 'My Plan' in the ChatGPT sidebar. Click on 'Manage my subscription' in the pop-up window.Select 'Cancel Plan'. Please note that your cancellation will take effect the day after the next billing date, and you can continue using our services until then. To avoid being charged for your next billing period, please cancel your subscription at least 24 hours before your next billing date. ## APIWe'll need to keep a payment method on file to account for any outstanding usage costs. You're welcome to cancel your pay-as-you-go service, by clicking 'Cancel paid account' in your billing overview. After the current month's invoice has been issued, the current card will no longer be charged. If you'd like to continue using the service, add a new payment method in the billing overview page and select 'Set as default'. You'll then be able to delete the old payment method.","1. Ask the customer to specify which service they are using. a. ""Could you please confirm whether you're using the ChatGPT Plus subscription or the API pay-as-you-go service?""2. Based on the customer's response: a. If the customer indicates they are using the ChatGPT Plus subscription, then: i. Inform the customer that they can cancel their ChatGPT Plus subscription to stop further charges at any time. - ""You can cancel your ChatGPT Plus subscription to stop further charges at any time."" ii. Provide the steps to cancel the subscription: - ""Click on 'My Plan' in the ChatGPT sidebar."" - ""Click on 'Manage my subscription' in the pop-up window."" - ""Select 'Cancel Plan'."" iii. Inform the customer about the cancellation effect date and billing details. - ""Please note that your cancellation will take effect the day after the next billing date, and you can continue using our services until then."" - ""To avoid being charged for your next billing period, please cancel your subscription at least 24 hours before your next billing date."" b. Else if the customer indicates they are using the API pay-as-you-go service, then: i. Inform the customer about the need to keep a payment method on file. - ""We need to keep a payment method on file to account for any outstanding usage costs."" ii. Provide the steps to cancel the pay-as-you-go service: - ""You can cancel your pay-as-you-go service by clicking 'Cancel paid account' in your billing overview."" iii. Inform the customer about the billing process after cancellation. - ""After the current month's invoice has been issued, the current card will no longer be charged."" iv. If the customer wants to continue using the service but delete the old payment method: - ""Add a new payment method in your billing overview page."" - ""Select 'Set as default' next to the new payment method."" - ""You'll then be able to delete the old payment method."" c. Else: i. Politely ask the customer to specify which service they are using to assist further. - ""I'm sorry, to assist you further, could you please specify whether you're using the ChatGPT Plus subscription or the API pay-as-you-go service?""3. Ask the customer if there is anything else you can assist with. - ""Is there anything else I can assist you with today?""4. Call the `case_resolution` function. `call the case_resolution function`**Function Definitions:**- `case_resolution` function: - **Purpose**: Finalize the case and mark it as resolved. - **Parameters**: None."
1,Business Associate Agreement,"How can I get a Business Associate Agreement (BAA) with OpenAI?Information about HIPAA compliance for healthcare companiesThe Health Insurance Portability and Accountability Act (HIPAA) is a U.S. federal law that requires privacy a